In [1]:
# augmenting ChatGPT using a knowledge graph and Lang Chain
# input: Question and Finalized set of triples 
# output: Answer from ChatGPT (Linked to ChatGPT-3.5 here)

# NUS-ISS MTECH EBAC Capstone Project (2023)
# Team Members: Kwan Yick Tan, Hiew Ming Yu, Lim Kim Hui, Chua Khai Shing, Ivan Ong Jun Yi

In [2]:
!pip install -q langchain
!pip install -q openai

In [3]:
import os
import openai
import gradio as gr
import networkx as nx
import matplotlib.pyplot as plt
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.indexes import GraphIndexCreator
from langchain.chains import GraphQAChain
from langchain.prompts import PromptTemplate
from langchain.graphs.networkx_graph import KnowledgeTriple

In [4]:
os.environ['OPENAI_API_KEY'] = '<OPENAI API KEY>' #To fill in OpenAI API KEY from user account

In [5]:
openai.api_key = os.environ['OPENAI_API_KEY']

In [6]:
# Initialize the knowledge graph and user question
kg = []  # Initial knowledge graph as a list of triples
graph = None  # Placeholder for the graph object
user_question = ""

In [7]:
# Create a function to update the knowledge graph and get answers
def update_graph_and_get_answer(user_question, triples):
    global kg, graph  # Use global variables

    # Parse the triples entered as a list of tuples
    new_triples = [tuple(map(str.strip, t.strip("()").split(","))) for t in triples.split("\n") if t.strip()]

    # Update the knowledge graph with the new triples
    kg.extend(new_triples)

    # Create or update the graph object
    if graph is None:
        graph = GraphIndexCreator(llm=OpenAI(temperature=0)).from_text('')
        for (node1, relation, node2) in kg:
            graph.add_triple(KnowledgeTriple(node1, relation, node2))
    else:
        for (node1, relation, node2) in new_triples:
            graph.add_triple(KnowledgeTriple(node1, relation, node2))

    # Generate a response using the current graph
    if user_question:
        chain = GraphQAChain.from_llm(OpenAI(temperature=0), graph=graph, verbose=True)
        answer = chain.run(user_question)
        return answer
    else:
        return "Please enter a question."

In [9]:
# Create the Gradio interface with text inputs
iface = gr.Interface(
    fn=update_graph_and_get_answer,
    inputs=[
        gr.inputs.Textbox(label="Enter your question:", type="text"),
        gr.inputs.Textbox(label="Enter additional triples (one per line):", type="text")
    ],
    outputs="text",
    title="NUS-ISS Capstone Team 1: GPT-3.5 Question Answering with Knowledge Graph",
    description="Enter your question and add additional triples (one per line) to update the knowledge graph.",
    live=True,
)

C:\Users\kwanyick\AppData\Local\Temp\ipykernel_44796\3557790534.py:5: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Textbox(label="Enter your question:", type="text"),
C:\Users\kwanyick\AppData\Local\Temp\ipykernel_44796\3557790534.py:5: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(label="Enter your question:", type="text"),
C:\Users\kwanyick\AppData\Local\Temp\ipykernel_44796\3557790534.py:5: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(label="Enter your question:", type="text"),
C:\Users\kwanyick\AppData\Local\Temp\ipykernel_44796\3557790534.py:6: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Textbox(label="Enter additional tripl

In [10]:
# REFER TO END OF CODE FOR QUESTION INPUT & ADDITIONAL TRIPLES INPUT

In [11]:
# Launch the first Gradio interface
display(iface.launch())

Running on local URL:  http://127.0.0.1:7867

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB

To create a public link, set `share=True` in `launch()`.




> Entering new GraphQAChain chain...
Entities Extracted:
 SMRT Corporation Ltd, Verztec Consulting Pte Ltd, Sunway Ship Supplies Pte Ltd, Grand Park City Hall, NS, COS
Full Context:


> Finished chain.


> Entering new GraphQAChain chain...
Entities Extracted:
 SMRT Corporation Ltd, Verztec Consulting Pte Ltd, Sunway Ship Supplies Pte Ltd, Grand Park City Hall, NS, COS
Full Context:
SMRT Corporation Ltd considers Certificate of Service (COS
SMRT Corporation Ltd supports NS
Verztec Consulting Pte Ltd considers Certificate of Service (COS
Verztec Consulting Pte Ltd supports NS
Sunway Ship Supplies Pte Ltd considers Certificate of Service (COS
Sunway Ship Supplies Pte Ltd supports NS
Grand Park City Hall considers Certificate of Service (COS
Grand Park City Hall supports NS

> Finished chain.


In [ ]:
# Knowledge graph visualization with triples

kg = [
('SMRT Corporation Ltd','considers','Certificate of Service (COS)'),
('Verztec Consulting Pte Ltd','considers','Certificate of Service (COS)'),
('Sunway Ship Supplies Pte Ltd','considers','Certificate of Service (COS)'),
('Grand Park City Hall','considers','Certificate of Service (COS)'),
('SMRT Corporation Ltd','considers','Certificate of Service (COS)'),
('Verztec Consulting Pte Ltd','supports','NS'),
('Sunway Ship Supplies Pte Ltd','supports','NS'),
('Grand Park City Hall','supports','NS'),
('SMRT Corporation Ltd','supports','NS'),
('Certificate of Service (COS)','provides','information'),
('Certificate of Service (COS)','considers','skills'),
('Certificate of Service (COS)','considers','attributes'),
('Certificate of Service (COS)','considers','personal attributes'),
('Certificate of Service (COS)','considers','skill sets'),
('Certificate of Service (COS)','considers','experiences'),
('Certificate of Service (COS)','considers','leadership qualities'),
('Certificate of Service (COS)','considers','experiences'),
('Certificate of Service (COS)','issued_to','NSFs'),
('Certificate of Service (COS)','issued_to','Nsmen'),
('Certificate of Service (COS)','issued_to','NS'),
]


index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

graph = index_creator.from_text('')
for (node1, relation, node2) in kg:
    graph.add_triple(KnowledgeTriple(node1, relation, node2))

In [ ]:
# Create directed graph
G = nx.DiGraph()
for node1, relation, node2 in kg:
    G.add_edge(node1, node2, label=relation)

# Plot the graph
plt.figure(figsize=(25, 25), dpi=300)
pos = nx.spring_layout(G, k=2, iterations=50, seed=0)

nx.draw_networkx_nodes(G, pos, node_size=5000, node_color='lightgreen')
nx.draw_networkx_edges(G, pos, edge_color='gray', edgelist=G.edges(), width=2)
nx.draw_networkx_labels(G, pos, font_size=12)
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=12)

# Display the plot
plt.axis('off')
plt.show()

In [ ]:
# THIS CELL IS NOT TO BE RUN 
# PURPOSE: INPUTS TO GRADIO (USER INTERFACE) FOR PROOF OF CONCEPT PURPOSE

# Qns to feed in to Gradio Above 

Do SMRT Corporation Ltd,Verztec Consulting Pte Ltd,Sunway Ship Supplies Pte Ltd,Grand Park City Hall consider NS Certificate of Service (COS)?

# triples to feed in to Gradio Above

(SMRT Corporation Ltd,considers,Certificate of Service (COS))
(Verztec Consulting Pte Ltd,considers,Certificate of Service (COS))
(Sunway Ship Supplies Pte Ltd,considers,Certificate of Service (COS))
(Grand Park City Hall,considers,Certificate of Service (COS))
(SMRT Corporation Ltd,considers,Certificate of Service (COS))
(Verztec Consulting Pte Ltd,supports,NS)
(Sunway Ship Supplies Pte Ltd,supports,NS)
(Grand Park City Hall,supports,NS)
(SMRT Corporation Ltd,supports,NS)
(Certificate of Service (COS),provides,information)
(Certificate of Service (COS),considers,skills)
(Certificate of Service (COS),considers,attributes)
(Certificate of Service (COS),considers,personal attributes)
(Certificate of Service (COS),considers,skill sets)
(Certificate of Service (COS),considers,experiences)
(Certificate of Service (COS),considers,leadership qualities)
(Certificate of Service (COS),considers,experiences)
(Certificate of Service (COS),issued_to,NSFs)
(Certificate of Service (COS),issued_to,Nsmen)
(Certificate of Service (COS),issued_to,NS)
